# KFIR EXPERIMENTATION

## SETUP: DEPENDENCIES AND CREDENTIALS

### Working Directory

What is the current working directory?:

In [3]:
import os
os.getcwd()

'U:\\PortableApps\\WinPython-32bit-3.6.1.0Qt5\\notebooks\\KFIR\\notebooks'

Add parent directory to path if necessary:

In [4]:
import sys, os, re

working_directory = os.getcwd()
if re.search('\\\\notebooks$', working_directory):
    one_directory_up = re.sub('\\\\notebooks$', '', working_directory)
    sys.path.append(one_directory_up)
    
sys.path

['',
 'U:\\PortableApps\\WinPython-32bit-3.6.1.0Qt5\\python-3.6.1\\python36.zip',
 'U:\\PortableApps\\WinPython-32bit-3.6.1.0Qt5\\python-3.6.1\\DLLs',
 'U:\\PortableApps\\WinPython-32bit-3.6.1.0Qt5\\python-3.6.1\\lib',
 'U:\\PortableApps\\WinPython-32bit-3.6.1.0Qt5\\python-3.6.1',
 'U:\\PortableApps\\WinPython-32bit-3.6.1.0Qt5\\python-3.6.1\\lib\\site-packages',
 'U:\\PortableApps\\WinPython-32bit-3.6.1.0Qt5\\python-3.6.1\\lib\\site-packages\\win32',
 'U:\\PortableApps\\WinPython-32bit-3.6.1.0Qt5\\python-3.6.1\\lib\\site-packages\\win32\\lib',
 'U:\\PortableApps\\WinPython-32bit-3.6.1.0Qt5\\python-3.6.1\\lib\\site-packages\\Pythonwin',
 'U:\\PortableApps\\WinPython-32bit-3.6.1.0Qt5\\python-3.6.1\\lib\\site-packages\\IPython\\extensions',
 'U:\\PortableApps\\WinPython-32bit-3.6.1.0Qt5\\settings\\.ipython',
 'U:\\PortableApps\\WinPython-32bit-3.6.1.0Qt5\\notebooks\\KFIR']

### Plotly

Read plotly credentials from file:

In [5]:
from preprocessor.Text_File import Text_File

plotly_file = Text_File('..//private//plotly_credentials')
plotly_file = plotly_file.return_content()
plotly_credentials = plotly_file.splitlines()

plotly_username = plotly_credentials[0]
plotly_key = plotly_credentials[1]

Import plotly:

In [6]:
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.tools import set_credentials_file

set_credentials_file(username=plotly_username, api_key=plotly_key)  # put your own plotly username and api key here 

### Gastrodon

For usage examples of Gastrodon, see [example dbpedia notebook](https://github.com/paulhoule/gastrodon/blob/master/notebooks/remote/Querying%20DBpedia.ipynb).

Import gastrodon (for running SPARQL queries in Jupyter):

In [7]:
from gastrodon import RemoteEndpoint,QName,ttl,URIRef,inline
import pandas as pd
pd.options.display.width=120
pd.options.display.max_colwidth=100

Define prefixes:

In [8]:
prefixes=inline("""
    @prefix wos: <http://wos.risis.eu/vocabulary/> .
    @prefix kfir: <http://clokman.com/kfir/ontology#> .
    @prefix ldr: <https://github.com/ali1k/ld-reactor/blob/master/vocabulary/index.ttl#> .
    @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
    @prefix wosGraph: <http://clokman.com/wos> .
    @prefix kfirGraph: <http://clokman.com/kfir> .
    
    @prefix dbo: <http://dbpedia.org/ontology/> .
""").graph

Read endpoint from file:

In [15]:
from preprocessor.Text_File import Text_File

# Get endpoint address from file
eculture_endpoint_url_file = Text_File('..//private//eculture_virtuoso_endpoint_address')
eculture_endpoint_url = eculture_endpoint_url_file.return_content()

Define endpoint:

In [16]:
eculture=RemoteEndpoint(
    url=eculture_endpoint_url,
    default_graph=None,
    prefixes=prefixes
)

### Triplicator

In [11]:
from retriever.sparql_tools import Sparql_Query

In [17]:
query = Sparql_Query()
query.set_endpoint(eculture_endpoint_url)

In [20]:
eculture=RemoteEndpoint(
    url=eculture_endpoint_url,
    default_graph=None,
    prefixes=prefixes,
)

## QUERIES

In [19]:
query.set_query("SELECT ?s ?p ?o WHERE {?s ?p ?o} LIMIT 5")
query.retrieve_results_from_endpoint()

{1: {'o': 'http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat',
  'p': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  's': 'http://www.openlinksw.com/virtrdf-data-formats#default-iid'},
 2: {'o': 'http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat',
  'p': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  's': 'http://www.openlinksw.com/virtrdf-data-formats#default-iid-nullable'},
 3: {'o': 'http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat',
  'p': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  's': 'http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank'},
 4: {'o': 'http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat',
  'p': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  's': 'http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank-nullable'},
 5: {'o': 'http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat',
  'p': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  's': 'http://www.openlinksw.com/virtrdf-data-fo

In [21]:
count=eculture.select("""
    SELECT (COUNT(*) AS ?count) { ?s ?p ?o .}
""").at[0,"count"]
count

71636021

In [22]:
count=eculture.select("""
SELECT (COUNT(DISTINCT ?article) as ?articles) 
WHERE{
    GRAPH wosGraph: {
        ?article a wos:Article .
    }
}
""").at[0,"articles"]
count

136125

In [26]:
from preprocessor.Text_File import Text_File

# Get endpoint address from file
eculture.select("""
    SELECT ?p (COUNT(*) AS ?count) { ?s ?p ?o .} GROUP BY ?p ORDER BY DESC(?count)
""")

,count
p,
rdf:type,10059030
rdfs:label,9664925
wos:CR,6672631
kfir:hasCited,5221917
http://www.w3.org/2004/02/skos/core#altLabel,3876744
wos:AF,3876692
kfir:hasAuthor,3485320
wos:DOI,2317751
ldr:annotations,2181336
